In [ ]:
import random, heapq, time, tracemalloc
#----------------------------------------UCS--------------------------------------
def is_goal_state(state):
    for i in range(len(state)):
        for j in range(i + 1, len(state)):
            if state[i] == state[j] or abs(state[i] - state[j]) == j - i:
                return False
    return True

def printBoardUCS(board):
    N = len(board)
    for row in range(N):
        for col in range(N):
            if board[col] == row:
                print('Q ', end='')
            else:
                print('* ', end='')
        print()

def SolveNQueensUCS(initial_board, N):
    pq = []
    heapq.heapify(pq)

    cost = 0
    heapq.heappush(pq, (cost, initial_board))

    while pq:
        cost, board = heapq.heappop(pq)
        if is_goal_state(board):
            return board
        row = cost
        for col in range(N):
            newboard = board.copy()
            newboard[row] = col
            newcost = cost + 1
            heapq.heappush(pq, (newcost, newboard))
    return []
#---------------------------------------------------------------------------------
#---------------------------------A*----------------------------------------------
class Board:
    def __init__(self, queens, conflicts, cost, block):
        self.queens = queens
        self.conflicts = conflicts
        self.cost = cost
        self.block = block

    def __lt__(self, other):
        return self.cost + self.conflicts < other.cost + self.conflicts

def calculateConflicts(queens):
    conflicts = 0
    for i in range(len(queens)):
        for j in range(i + 1, len(queens)):
            if i != j and queens[i] == queens[j] or abs(queens[i] - queens[j]) == j - i:
                conflicts += 1
    return conflicts

def generateSuccessors(current, n):
    successors = []
    for row in range(current.block + 1, n):
        for i in range(n):
            col = i
            newQueens = current.queens.copy()
            newQueens[row] = col
            newCost = current.cost + 1
            conflicts = calculateConflicts(newQueens)
            successors.append(Board(newQueens, conflicts, newCost, row))
    return successors

def SolveNQueensAStar(initial_board, n):
    pq = []
    h = calculateConflicts(initial_board)
    heapq.heappush(pq, Board(initial_board, h, -1, -1))
    while pq:
        current = heapq.heappop(pq)
        if current.conflicts == 0:
            return current.queens
        successors = generateSuccessors(current, n)
        for successor in successors:
            heapq.heappush(pq, successor)
    return []
def printBoardAStar(queens):
    n = len(queens)
    for row in range(n):
        for col in range(n):
            if queens[col] == row:
                print("Q ", end="")
            else:
                print("* ", end="")
        print()
#---------------------------------------------------------------------------------
#---------------------------------------Genetic algorithm-------------------------

def fitness(cur_state):
        conflicts = 0
        for i in range(len(cur_state)):
            for j in range(i + 1, len(cur_state)):
                if cur_state[i] == cur_state[j] or abs(cur_state[i] - cur_state[j]) == j - i:
                    conflicts += 1
        return conflicts

def initialize_population(n):
        pop_list = []
        num = random.randint(2, n)
        while num != 0:
            temp = [random.randint(0, n - 1) for _ in range(n)]
            heapq.heappush(pop_list, (fitness(temp), temp))
            num -= 1
        return pop_list

def goal_test(population: list):
        for i in population:
            if fitness(i[1]) == 0:
                return i[1]
        return None

def random_pick(population):
        new_population = []
        n = random.randint(2, len(population))
        for i in range(n):
            new_population.append(population[i][1])
        return new_population

def crossover(parent1, parent2):
        p_len = len(parent1)
        cross_point = random.randint(0, p_len - 1)
        return parent1[:cross_point] + parent2[cross_point:p_len], parent2[:cross_point] + parent1[cross_point:p_len]

def mutate(state):
        state_len = len(state)
        mutate_point = random.randint(0, state_len - 1)
        mutation = random.randint(0, state_len - 1)
        state[mutate_point] = mutation
        return state

def SolveNQueensGA(init_state, n):
        population = initialize_population(n)
        heapq.heappush(population, (fitness(init_state), init_state))
        result = goal_test(population)

        while result == None:
            random_pop = random_pick(population)
            for i in range(0, len(random_pop), 2):
                if i + 2 <= len(random_pop):
                    #Crossover
                    random_pop[i], random_pop[i + 1] = crossover(random_pop[i], random_pop[i + 1])
                    #mutate
                    if random.random() <= 0.1:
                        random_pop[i] = mutate(random_pop[i])
                    if random.random() <= 0.1:
                        random_pop[i + 1] = mutate(random_pop[i + 1])

            for i in range(len(random_pop)):
                population.pop()
            for board in random_pop:
                heapq.heappush(population, (fitness(board), board))
            result = goal_test(population)
        return result

def printBoardGA(queens):
    n = len(queens)
    for row in range(n):
        for col in range(n):
            if queens[col] == row:
                print("Q ", end="")
            else:
                print("* ", end="")
        print()
#---------------------------------------------------------------------------------
if __name__ == "__main__":
    N = int(input("Enter the size of the chessboard (N): "))
    initial_board = [random.randint(0, N - 1) for _ in range(N)]
    print("1. UCS")
    print("2. A*")
    print("3. Genetic algorithm")
    choice = int(input("Your choice: "))
    t = []
    mem = []
    NUM_RUN_TIME = 3
    for i in range(NUM_RUN_TIME):
        tracemalloc.start() #start tracking memory usage
        start_time = time.time() #start tracking running time

        if choice == 1:
            solution = SolveNQueensUCS(initial_board, N)
        if choice == 2:
            solution = SolveNQueensAStar(initial_board, N)
        if choice == 3:
            solution = SolveNQueensGA(initial_board, N)

        peak = tracemalloc.get_traced_memory()[1]
        tracemalloc.stop()
        t.append(time.time() - start_time)
        mem.append(peak / 1024**2)

    if choice == 1:
        if not solution:
            print("No solution found.")
        else:
            print("Solution found:")
            printBoardUCS(solution)
    if choice == 2:
        if not solution:
            print("No solution found.")
        else:
            print("Solution found:")
            printBoardAStar(solution)
    if choice == 3:
        if not solution:
            print("No solution found.")
        else:
            print("Solution found:")
            printBoardGA(solution)

    print(f"Avarage running time: {sum(t) / NUM_RUN_TIME:.4f} seconds")
    print(f"Memory usage: {sum(mem) / NUM_RUN_TIME:.2f} MB")



: 